In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

import argparse
import logging
import pandas as pd
import random
import os
import numpy as np
from loggers import logger

from process import input_data, preprocess

def set_logger(log_name):
    log_obj = logger.AutoMLLog(log_name)
    log_obj.set_handler('automl_process')
    log_obj.set_formats()
    auto_logger = log_obj.addOn()
    
    auto_logger.info('logger 세팅')

In [2]:
#데이터 인코딩 문제

In [5]:
log_name = 'practice'
set_logger(log_name)
data, var_list, num_var, obj_var = input_data.Data_load('storage/Walmart.csv', log_name).read_data()
df = preprocess.Preprocessing(log_name, data, var_list, num_var, obj_var, target='Weekly_Sales', date_var= 'Date', store_list=['Store'], unit='week').df

logger 세팅
csv 데이터 불러오기
storage/Walmart.csv
데이터 구성: 6435 행, 8열
Memory usage of dataframe is 0.39MB
Memory usage after optimization is: 0.10MB
Decreased by 73.7%
결측치 처리
결측치 처리 이후 데이터 구성: 6435 행, 8열
전처리를 위한 target, date, store 분리
정규화 진행
라벨 인코딩 진행
전처리 완료


   Temperature  Fuel_Price       CPI  Unemployment  Holiday_Flag  \
0    -0.995605   -1.712891  1.004883      0.058716     -0.274170   
1    -1.202148   -1.763672  1.007812      0.058716      3.646484   
2    -1.124023   -1.840820  1.007812      0.058716     -0.274170   
3    -0.761230   -1.738281  1.010742      0.058716     -0.274170   
4    -0.768066   -1.597656  1.010742      0.058716     -0.274170   

   Weekly_Sales        Date  Store  
0   1643690.875  05-02-2010      1  
1   1641957.500  12-02-2010      1  
2   1611968.125  19-02-2010      1  
3   1409727.625  26-02-2010      1  
4   1554806.625  05-03-2010      1  
시계열용 전처리 진행
시계열 전처리 후 df
   Temperature  Fuel_Price       CPI  Unemployment  Holiday_Flag  \
0     0.607910   -1.644

In [7]:
train_test = pd.DataFrame()
train_test['ds'] = df['Date']
train_test['y'] = 0

In [10]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import optuna

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [9]:

prophet = Prophet(seasonality_mode='multiplicative', 
                  yearly_seasonality=False,
                  weekly_seasonality=False, daily_seasonality=False)
prophet.fit(x_df)

NameError: name 'x_df' is not defined